In [2]:
from collections import Counter
# К ближайших соседей

def raw_majority_vote(labels):
    """
    Грубый подсчет большинства голосов
    """
    votes = Counter(labels)
    winner, _ = votes.most_common(1)[0]
    
    return winner


def majority_vote(labels):
    """
    Подразумевает, что метки упорядочены от ближайшей до самой удаленной
    """
    vote_counts = Counter(labels)
    winner, winner_count = votes.most_common(1)[0]
    num_winners = len([count for count in vote_counts.values() if count == winner_count])
    # если победитель один - возвращаем его     
    if num_winners == 1:
        return winner
    
    return majority_vote(labels[:-1])


def knn_classify(k, labeled_points, new_point):
    """
    каждая маркированная точка должна быть представлена парой (точка, метка)
    """
#     TODO: переписать по человечески!!!
    # упорядочить маркированные точки, от ближайшей до самой удаленной     
#     by_distance = sorted(labeled_points, key=lambda(point, _): distance(point, new, point))
    
    # найти метки для k-ближайших
    k_nearest_labels = [label for _, label in by_distance[:k]]
    
    # и дать им проголосовать
    return majority_vote(k_nearest_labels)